In [4]:
import math
import numpy as np

# Method interfaces

In [23]:
class IterativeMethod:
    def compute(self, A: [[float]], b: [float], tolerance: float, max_iteration_count: int) -> ([float], int, bool):
        raise AssertionError("Not implemented yet")

In [2]:
class ForwardMethod:
    def compute(self, A: [[float]], b: [float]) -> [float]:
        raise AssertionError("Not implemented yet")

# Methods

In [32]:
class GaussMethod(ForwardMethod):
    @staticmethod
    def _find_max_row(A: [[float]], ind: int) -> int:
        cur_max = A[ind, ind]
        cur_ind = ind
        for i in range(ind, len(A)):
            cur_val = A[i, ind]
            if cur_max < cur_val:
                cur_max = cur_val
                cur_ind = i
        return cur_ind
    
    @staticmethod
    def normalize_line(A: [[float]], ind: int):
        a = A[ind, ind]
        for j in range(ind, len(A)):
            A[ind, j] = A[ind, j] / a
            
    @staticmethod
    def go_down(A: [[float]], b: [float], ind: int):
        for i in range(ind + 1, len(A)):
            main_val = A[i, ind]
            b[i] -= main_val * b[ind]
            for j in range(ind, len(A)):
                A[i, j] -= main_val * A[ind, j]
            
    
    def compute(self, A: [[float]], b: [float]) -> [float]:
        # make matrix diagonal
        for i in range(len(A)):
            # find row with max value in 'i' column -- main row
            j = GaussMethod._find_max_row(A, i)
            # swap main row and current
            A[[i, j]] = A[[j, i]]
            
            b[i] /= A[i, i]
            GaussMethod.normalize_line(A, i)
            GaussMethod.go_down(A, b, i)
        # get solution
        ans = np.zeros(len(A))
        for i in reversed(range(len(A))):
            ans[i] = b[i]
            for j in reversed(range(i + 1, len(A))):
                ans[i] -= A[i][j] * ans[j]
        print(ans)
        

def test():
    A = np.array([[10,1,-1],[1,10,-1],[-1,1,10]], dtype=np.float64)
    b = np.array([11,10,10], dtype=np.float64)
    gauss = GaussMethod()
    gauss.compute(A, b)
    
test()    

[1.1020202  0.99090909 1.01111111]


In [2]:
# same lenght
def deltaVectorsNorm(x, y):
    res = 0.0
    for i in range (0, len(x)):
        res += (y[i] - x[i]) ** 2
    return math.sqrt(res)

In [3]:
# Ax = b
# epsiolon - accuracy parameter
# method -
#   0 - Jacobi
#   1 - Seidel (determinant(B) should be less 1)
#   2 - Seidel with relaxation
# wRelax - relaxation parameter
def templateJakobiAndSeidel(A, b, epsilon, method, wRelax):
    iterations = 0
    maxIterations = 100
    B = []
    d = []
    xn = []
    x = []
    
    for i in range (0, len(A)):
        d.append(b[i] / A[i][i])
        xn.append(d[i])
        x.append(d[i] + 1000)
        B.append([])
        for j in range (0, len(A[i])):
            B[i].append(-A[i][j] / A[i][i])
        B[i][i] = 0.0
    
    # detB = determinant(B)
    # eps = (1 - detB) * epsilon / detB
    eps = epsilon
    
    while ((deltaVectorsNorm(x, xn) > eps) and iterations < maxIterations):
        x = xn.copy()
        for i in range (0, len(B)):
            xn[i] = 0.0
            for j in range (0, len(B[i])):
                xn[i] += B[i][j] * (xn[j] if (j < i and method > 0) else x[j])
            xn[i] += d[i]
            if (method == 2):
                xn[i] = wRelax * xn[i] + (1 - wRelax) * x[i]
        iterations = iterations + 1
            
    
    return (xn, iterations)

In [4]:
def jakobi(A, b, epsilon):
    return templateJakobiAndSeidel(A, b, epsilon, 0, 0) 

In [5]:
def seidel(A, b, epsilon):
    return templateJakobiAndSeidel(A, b, epsilon, 1, 0) 

In [6]:
def seidelRelax(A, b, epsilon, wRelax):
    return templateJakobiAndSeidel(A, b, epsilon, 2, wRelax) 

In [7]:
def test_jakobi():
    A = [[10,1,-1],[1,10,-1],[-1,1,10]]
    b = [11,10,10]
    print ("Jakobi")
    (ans, it) = jakobi(A, b, 0.000001)
    print (ans)
    print (it)

test_jakobi()

Jakobi
[1.1020202000000001, 0.9909091, 1.0111111]
6


In [8]:
def test_seidel():
    A = [[10,1,-1],[1,10,-1],[-1,1,10]]
    b = [11,10,10]
    print ("Seidel")
    (ans, it) = seidel(A, b, 0.000001)
    print (ans)
    print (it)

test_seidel()

Seidel
[1.1020201985951001, 0.99090909050459, 1.011111110809051]
5


In [9]:
def test_seidel_relax():
    A = [[10,1,-1],[1,10,-1],[-1,1,10]]
    b = [11,10,10]
    print ("Seidel with relaxation")
    (ans, it) = seidelRelax(A, b, 0.000001, 0.08)
    print (ans)
    print (it)

test_seidel_relax()

Seidel with relaxation
[1.1020120682451873, 0.9909105605032578, 1.0111016830698611]
94
